In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Volume Estimation and Uncertainty Analysis

### 2D Volume Estimation

In petrophysics, 2D volume estimation is used to calculate the volume of hydrocarbons in a reservoir using two-dimensional data, typically from seismic surveys or well logs. Here's a simplified process:

1. **Data Collection**: Gather 2D seismic data and well log data. Seismic data provides a cross-sectional view of the subsurface, while well logs give detailed information about the rock and fluid properties at specific locations.

2. **Structural Mapping**: Create structural maps of the reservoir. This involves interpreting seismic data to delineate the boundaries and structure of the reservoir.

3. **Reservoir Zoning**: Divide the reservoir into zones based on geological and petrophysical properties. Each zone may have different characteristics affecting hydrocarbon volume.

4. **Property Assignment**: Assign petrophysical properties (e.g., porosity, water saturation) to each zone. These properties are derived from well logs and core samples.

5. **Volume Calculation**: Calculate the volume of each zone using the formula:
   $$
   V = A \times h \times \phi \times (1 - S_w)
   $$
   where:
   - \( V \) is the hydrocarbon volume.
   - \( A \) is the area of the zone.
   - \( h \) is the thickness of the zone.
   - \( \phi \) is the porosity.
   - \( S_w \) is the water saturation.

### Uncertainty Analysis

Uncertainty analysis is crucial because it helps quantify the confidence in the volume estimates. Here are the key steps:

1. **Identify Uncertainties**: Determine the sources of uncertainty, such as data quality, interpretation errors, and variability in petrophysical properties.

2. **Quantify Uncertainties**: Use statistical methods to quantify the uncertainties. This can involve:
   - **Monte Carlo Simulation**: Generate multiple realizations of the reservoir model by varying the input parameters within their uncertainty ranges.
   - **Sensitivity Analysis**: Assess how changes in each parameter affect the volume estimate.

3. **Probabilistic Volume Estimation**: Combine the results from the simulations to create a probabilistic distribution of the hydrocarbon volume. This provides a range of possible volumes with associated probabilities.

4. **Risk Assessment**: Evaluate the risk associated with different volume estimates. This helps in making informed decisions about reservoir development and management.

quick_pp estimates the volume by considering the distribution of each parameter. Smaller range and smaller standard deviation indicates less uncertainty and vice versa applies.

Monte Carlo simulation is implemented ...

In [ ]:
import pandas as pd

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [ ]:
df = project.get_all_data()

### Fluid Contact Analysis

In [ ]:
from quick_pp.plotter import stick_plot, generate_well_config, generate_zone_config

zone_config = generate_zone_config()
print(zone_config)

In [ ]:
well_names = project.get_well_names()
well_config = generate_well_config(well_names)
print(well_config)

In [ ]:
from quick_pp.plotter import update_well_config

for well in well_names:
    fluid_dict = {
        'GUT': 7900,
        'GDT': 8300,
        'GOC': 8300,
        'OUT': 8300,
        'OWC': 8600,
        'ODT': 8600,
        'WUT': 8600,
    }
    well_config = update_well_config(well_config, well, "ALL", fluid_dict)

In [ ]:
sorting_dict = {
    'HW-10': 8,
    'HW-24': 9,
    'HW-25': 10,
    'HW-26': 11,
    'HW-27': 12,
    'HW-28': 13,
    'HW-29': 14,
    'HW-3': 1,
    'HW-30': 15,
    'HW-31': 16,
    'HW-32': 17,
    'HW-4': 2,
    'HW-5': 3,
    'HW-6': 4,
    'HW-7': 5,
    'HW-8': 6,
    'HW-9': 7
}

for well, sorting in sorting_dict.items():
    well_config = update_well_config(well_config, well, sorting=sorting)

In [ ]:
stick_plot(df, well_config)

### Statistical Summary

In [ ]:
import matplotlib.pyplot as plt

for well, data in df.groupby('WELL_NAME'):
    plt.figure(figsize=(15, 2))
    plt.title(well)
    plt.plot(data['DEPTH'], data['SWT'], label='SWT')
    plt.plot(data['DEPTH'], data['SW'], label='SW')
    plt.legend()
    plt.ylim(0, 1)

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

# df = project.get_all_data()

df['VSHALE'] = df['VCLW']
df['SWT'] = df['SWT'].where(df['SWT'] <= 1, 1)
df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)

In [ ]:
from quick_pp.qaqc import extract_quick_stats

extract_quick_stats(compare_df)

### Volume Estimation

In [ ]:
from quick_pp.ressum import mc_volumetric_method

area_bound = (120, 840, 660, 160)
thickness_bound = (3, 9, 6, 3)
porosity_bound = (.065, .255, .175, .085)
water_saturation_bound = (.230, 1, .186)
volume_factor_bound = (1.2, 1.4)
recovery_factor_bound = (.1, .5, .3, .075)

n_try = 10000
reserves = mc_volumetric_method(
    area_bound=area_bound,
    thickness_bound=thickness_bound,
    porosity_bound=porosity_bound,
    water_saturation_bound=water_saturation_bound,
    volume_factor_bound=volume_factor_bound,
    recovery_factor_bound=recovery_factor_bound,
    n_try=n_try, percentile=[10, 50, 90]
)

### Sensitivity Analysis

In [ ]:
from quick_pp.ressum import sensitivity_analysis

sensitivity_analysis(
    area_bound=area_bound,
    thickness_bound=thickness_bound,
    porosity_bound=porosity_bound,
    water_saturation_bound=water_saturation_bound,
    volume_factor_bound=volume_factor_bound,
    recovery_factor_bound=recovery_factor_bound
)